#Data 09: Restaurant

Zomato는 인도의 맛집/음식 배달 웹 사이트이다.  
이 데이터는 2019 년 3 월 15 일까지 zomato 웹 사이트에서 제공되는 데이터를 담고 있다.
*   Data from: https://www.kaggle.com/himanshupoddar/zomato-bangalore-restaurants

##1.데이터 둘러보기

In [ ]:
#한글 폰트 설정하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
#런타임 다시 시작

In [ ]:
#기본 패키지 불러오기
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('seaborn')
sns.set(font_scale=1)
plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['font.family'] = 'NanumGothic'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#구글 드라이브 마운트하기(이미 되어있다면 건너뛴다)
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
#데이터 불러오기
#드라이브에 올려준 csv 데이터 파일 불러오기
zmt = pd.read_csv('/content/drive/MyDrive/Python/09_Restaurant/zomato.csv') #~16 sec

In [ ]:
#불러온 데이터 일단 보기
zmt.head(3)

<컬럼 설명>  
* url : Zomato 사이트의 URL
* address : 식당 주소
* name : 식당 이름
* online_order : 온라인 주문 가능 여부(Yes/No)
* book_table : 테이블 예약 가능 여부(Yes/No)
* rate : 식당 평점(5점 만점)
* votes : 평점 참여 횟수
* phone : 식당 전화번호
* location : 식당 위치
* rest_type : 레스토랑 타입
* dish_liked : 인기 메뉴
* cuisines : 메뉴 타입
* approx_cost(for two people) : 대략적인 가격(두 사람 기준)
*reviews_list : 평점과 리뷰로 구성된 후기 정보
* menu_item : 레스토랑에서 주문 가능한 메뉴
* listed_in(type) : 식사 종류
* listed_in(city) : 레스토랑 위치

In [ ]:
#데이터 기본 정보 확인하기
zmt.info()

In [ ]:
#데이터 내용 간단히 정리해서 출력하기
print("총 데이터 개수: ", zmt.shape[0]*zmt.shape[1])
print("총 결측치 수: {} = 전체 데이터의 {:.2f}% ".format(zmt.isnull().sum().sum(), (zmt.isnull().sum().sum()*100)/(zmt.shape[0]*zmt.shape[1])))
print("전체 식당 수: ", zmt['name'].nunique())

##2.데이터를 보고 질문 만들기

* 방갈로르에 가장 많은 레스토랑은?  
* 방갈로르의 레스토랑 운영 형태는?  
* 방갈로르의 어느 지역에 레스토랑이 가장 많은가?  
* 온라인 주문/테이블 예약이 금액과 평점에 영향을 주는가?  
* 레스토랑의 종류 별로 메뉴가 다를까?
* 레스토랑 리뷰를 분석할 수 있을까?

##3.데이터 정비하기

###3-1. 필요없는 컬럼 삭제

In [ ]:
zmt = zmt.drop(['url','phone', 'menu_item'],axis = 1)
zmt.head()

###3-2. 중복 데이터 처리하기

In [ ]:
#중복 데이터 확인하기
zmt.duplicated().sum()

In [ ]:
#중복 데이터 삭제하기
zmt.drop_duplicates(inplace = True)
zmt.info()

###3-3. NaN 처리하기

In [ ]:
#컬럼별로 NULL 개수 확인하기
zmt.isnull().sum()

In [ ]:
#Nan이 존재하는 행은 다 삭제하기
zmt.dropna(how = 'any',inplace = True)
zmt.info()

###3-4. 컬럼 정비하기

In [ ]:
#컬럼명 확인하기
zmt.columns

In [ ]:
#컬럼명 변경하기 
zmt = zmt.rename(columns={'approx_cost(for two people)':'cost',
                                'listed_in(type)':'type',
                                'listed_in(city)':'city'})
zmt.columns

In [ ]:
#cost 컬럼에 ,가 들어가 있어서 object 타입이다. 변경해주자
print(zmt['cost'].unique())

In [ ]:
zmt['cost'] = zmt['cost'].astype(str) #string으로 변환하기
zmt['cost'] = zmt['cost'].replace(',','',regex = True) #콤마 삭제하기
zmt['cost'] = zmt['cost'].astype(float) #데이터타입을 float으로 변경하기
zmt['cost'].unique()

In [ ]:
#rating 컬럼 정비하기
print(zmt['rate'].unique())

In [ ]:
zmt['rate'] = zmt['rate'].astype(str) #string으로 변환하기
zmt['rate'] = zmt['rate'].replace('/5','',regex = True) #/5 -> 삭제
zmt['rating'] = zmt['rate'].replace('NEW','-',regex = True) #NEW > 0

zmt = zmt.loc[zmt['rate'] !='NEW'].reset_index(drop=True)

In [ ]:
zmt['rating'] = zmt['rating'].replace('-','0',regex = True) #rating 컬럼은 숫자 타입으로 바꿔주기 위해 한번 더 변환
zmt['rating'] = zmt['rating'].astype(float) #float으로 변환

In [ ]:
zmt.info()

In [ ]:
zmt.head(5)

In [ ]:
#online_order와 book_table 컬럼의 Yes -> 1, No -> 0으로 바꿔주기
zmt['online_order'] =zmt['online_order'].map({"Yes":1,"No":0}).astype(int)
zmt['book_table']=zmt['book_table'].map({"Yes":1,"No":0}).astype(int)
zmt.head()

##4.EDA & Visualization

In [ ]:
#매장 수가 가장 많은 레스토랑은?
f, ax = plt.subplots(1, 1, figsize = (12,6))

sns.barplot(x= zmt['name'].value_counts()[:10],y = zmt['name'].value_counts()[:10].index, palette = 'mako')
plt.title('방갈로르에 매장이 많은 레스토랑 Top 10', size=18)
plt.xlabel('매장 수')
plt.ylabel('레스토랑 이름')

In [ ]:
#레스토랑 운영 형태
plt.figure(figsize = (12,6))

sns.barplot(x= zmt['type'].value_counts()[:15],y = zmt['type'].value_counts()[:15].index,palette = 'icefire')
plt.title('방갈로르의 레스토랑 운영 형태', size=18)
plt.xlabel('매장 수')
plt.ylabel('레스토랑 타입')

In [ ]:
#방갈로르의 위치 별 레스토랑 수
f, ax = plt.subplots(1, 1, figsize = (12,12))

sns.countplot(zmt['city'], ax=ax)
sns.countplot(zmt['city']).set_xticklabels(sns.countplot(zmt['city']).get_xticklabels(), rotation=45, ha="right")
plt.title('방갈로르의 위치 별 레스토랑 수', size=18)
plt.xlabel('')

In [ ]:
#온라인 주문과 예약이 가능한 식당 비율
f, ax = plt.subplots(1, 2, figsize = (13,5))

sns.countplot(zmt['online_order'],palette = 'dark:salmon_r', ax=ax[0])

ax[0].set_title('온라인 주문 가능 매장')
ax[0].set_ylabel('Number of outlets') 

sns.countplot(zmt['book_table'],palette = 'YlOrBr', ax=ax[1])
ax[1].set_title('테이블 예약 가능 매장')
ax[1].set_ylabel('') 

In [ ]:
#온라인 주문과 테이블 예약과 평균 금액과의 관계
f, ax = plt.subplots(1, 2, figsize = (13,5))

sns.boxplot(x=zmt['online_order'], y=zmt['cost'], palette = 'dark:salmon_r', ax=ax[0])
ax[0].set_title('온라인 주문 가능 매장', size=15)
ax[0].set_ylabel('평균 식사 금액(2인 기준)') 
ax[0].set_ylim(0, 2000)

sns.boxplot(x=zmt['book_table'], y=zmt['cost'], palette = 'YlOrBr', ax=ax[1])
ax[1].set_title('테이블 예약 가능 매장', size=15)
ax[1].set_ylabel('') 
ax[1].set_ylim(0, 2000)

In [ ]:
#온라인 주문과 테이블 예약과 평점과의 관계
f, ax = plt.subplots(1, 2, figsize = (13,5))

sns.boxplot(x=zmt['online_order'], y=zmt['rating'], palette = 'dark:salmon_r', ax=ax[0])
#ax[0].set_title('온라인 주문 가능 매장', size=15)
#ax[0].set_ylabel('평점') 

sns.boxplot(x=zmt['book_table'], y=zmt['rating'],  palette = 'YlOrBr', ax=ax[1])
#ax[1].set_title('테이블 예약 가능 매장', size=15)
#ax[1].set_ylabel('') 


In [ ]:
#온라인 주문과 예약이 가능하면 평점이 더 높을까?
pd.crosstab(zmt['rate'],zmt['online_order'])
a = pd.crosstab(zmt['rate'],zmt['online_order'])

In [ ]:
pd.crosstab(zmt['rate'],zmt['book_table'])
b = pd.crosstab(zmt['rate'],zmt['book_table'])

In [ ]:
#stacking bar 그래프로 나타내기
f, ax = plt.subplots(2, 1, figsize = (13,5))

sns.set_style('white')

a.plot(kind="bar",stacked=True, ax=ax[0])
ax[0].xaxis.set_visible(False)
ax[0].set_title('rate', size=18)
plt.ylabel('') 
plt.xlabel('')


b.plot(kind="bar",stacked=True, ax=ax[1])
plt.ylabel('') 
plt.xlabel('Ratings')

##5.텍스트 클라우드

In [ ]:
#패키지 삽입
from wordcloud import WordCloud, STOPWORDS 

In [ ]:
zmt_re = zmt.copy()

In [ ]:
#상위 3개 레스토랑 타입에 대해서 워드 클라우드를 그리자
rest=zmt_re['rest_type'].value_counts()[:3].index 

In [ ]:
#레스토랑 타입별로 워드클라우드 그리는 함수

def zomato_wordcloud(rest):
    plt.figure(figsize = (20,20))
    for i, rest_tp in enumerate(rest):
        plt.subplot(1,3,i+1)
        dishes = ''
        data = zmt_re[zmt_re['rest_type'] == rest_tp]
        #dish_liked에 있는 단어들을 쪼개준다
        for word in data['dish_liked']:
            words=word.split()
            dishes=dishes+ " ".join(words)+" "
        wordcloud = WordCloud(background_color='white', colormap = 'seismic', collocations=False, stopwords = stopwords, width=1200, height=1200).generate(dishes)
        plt.imshow(wordcloud)
        plt.title(rest_tp)
        plt.axis("off")


In [ ]:
stopwords = set(STOPWORDS) 
zomato_wordcloud(rest)

##6.간단한 감정분석(Sentiment Analysis)
 감정 분석은 텍스트 분석 기술을 통해, 텍스트 데이터 내에서 감정(긍정적, 부정적 및 중립적)을 해석하고 분류하는 방법입니다.  
   감정 분석 패키지에도 여러가지가 있는데, 여기서는 TextBlob 패키지를 활용하겠습니다.   
 감정은 미리 단어 사전에 분류되어진 값을 사용하는데,     
 polarity 값이 양의 값을 가질 경우 긍정, 음의 값을 가지면 부정을 의미하며 0일 경우 중립 단어입니다.

In [ ]:
from textblob import TextBlob

In [ ]:
#리뷰를 감정 분석한 새로운 컬럼(review_sent)을 만든다(~8분)

sent_res = []
for i in range(0, len(zmt)):
    analysis = TextBlob(zmt.loc[i,'reviews_list']) #리뷰의 감정을 분석한다
    if analysis.sentiment.polarity > 0: 
        sent_res.append('positive')
    elif analysis.sentiment.polarity == 0:
        sent_res.append('neutral')
    else:
        sent_res.append('negative')
zmt['review_sent'] = sent_res

In [ ]:
zmt.head()

In [ ]:
#plotly로 비용과 평점 사이의 상관관계 나타내고 감정분석 결과를 나눠서 보자
from plotly.subplots import make_subplots
import plotly.express as px

#원의 크기는 투표 수로 나타낸다!
fig = px.scatter(zmt, x="rate", y="cost", size="votes", color='review_sent'
                     ,log_x=True, size_max=60)
fig.show()

##6.Review

* 방갈로르에 가장 많은 레스토랑은?  
* 방갈로르의 레스토랑 운영 형태는?  
* 방갈로르의 어느 지역에 레스토랑이 가장 많은가?  
* 온라인 주문/테이블 예약이 금액과 평점에 영향을 주는가?  
> 테이블 예약 가능 매장인 경우 금액대도 높고 평점도 좋았다.  
온라인 주문 여부는 금액과 평점에 큰 차이를 끼치지 않았다.

* 레스토랑의 종류 별로 메뉴가 다를까?
> 텍스트 클라우드로 메뉴를 분석했다
* 레스토랑 리뷰를 분석할 수 있을까?
> 감정분석을 통해 리뷰의 긍정/부정/중립을 나누어 볼 수 있었다